In [4]:
# LEAFToolbox-SL2P CCRS
#
# Applies the LEAF-ToolBox-SL2P with CCRS networks to process input images on Googe Earth Engine
# AND exports to google drive.
# 
# Note: This is an exact modified version of LEAF-ToolBox-SL2P  for image by image products
# The modification involves using land cover specific networks that in some cases (crops, grasses) are exactly the same
# as the network in Weiss and Baret,, 2016 used in LEAF-Toolbox-SL2P but for forests and shrubs are different 
# as theya re calibrated with simulations based on PROSAIL with different prior parameters including clumping.
#
# You will need a Google Earth Engine Account linked to a Google Account with sufficient
# disk space for the output.
#
# Refer to https://github.com/rfernand387/LEAF-Toolbox/Source-Python-GBOV/Validation-LeafToolbox-SL2PnomatchCCRSGBOV-v4.ipynb
# for configuration of anaconda environment.

In [273]:
import ee
import folium
import time
from folium import plugins
import numpy as np
import matplotlib.pyplot as plt
import geopy as gp
import math 

In [6]:
ee.Authenticate()

Enter verification code:  4/1AY0e-g78AwAnPdso_n0WTfJJOQfKucVR6l-AxI8halwjY271MzHcpOlgZw0



Successfully saved authorization token.


In [283]:
# Authorizes the script with Earth Engine using the credentials.json file. Ensure either ee.Authenticate() has been called in a previous
# script or the "$ earthengine authenticate" command has been run in the environment to set up the credentials.json file
# The google account you provide during the authentication flow must have received access to Google Earth Engine
ee.Initialize()

In [284]:
def check_ee_tasks(ee_tasks: list = []):
    for task in ee_tasks:
        taskStatus = ee.data.getTaskStatus(task.id)[0]
        print(taskStatus["description"] + ": " + taskStatus["state"])


In [285]:
# Wait loop for Earth Engine tasks to complete. Polls for the task status the specificed number of seconds until it is no longer active
def task_wait_loop(ee_task, wait_interval):
    print(ee.data.getTaskStatus(ee_task.id)[0]["description"]+":", end=" ")
    prev_task_status = ee.data.getTaskStatus(ee_task.id)[0]["state"]
    print(prev_task_status, end=" ")
    while ee_task.active():
        task_status = ee.data.getTaskStatus(ee_task.id)[0]["state"]
        if(task_status != prev_task_status):
            print(task_status, end=" ")
        prev_task_status = task_status
        time.sleep(wait_interval)
    print(ee.data.getTaskStatus(ee_task.id)[0]["state"])

In [286]:
# Create a folium map object.
def displayImage(image,minVal,maxVal) :
    my_map = folium.Map(location=[20, 0], zoom_start=3, height=500)
    vis_params = {
      'min': minVal,
      'max': maxVal}
    my_map.add_ee_layer(image, vis_params, 'Image')
    # Add a layer control panel to the map.
    my_map.add_child(folium.LayerControl())

    # Add fullscreen button
    plugins.Fullscreen().add_to(my_map)

    # Display the map.
    display(my_map)

In [287]:
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [288]:
default_asset_location = "users/hemitshah"
def export_collection_to_gee(collection, num_images: int = 0, image_names: list = [], asset_folder: str = default_asset_location, scale: float = 20, max_pixels: int = 1e8, data_type: str = "float"):
    
    collection = ee.ImageCollection(collection)
    
    image_list = collection.toList(num_images)
    task_list = []
    
    for i in range(num_images):
        image = ee.Image(image_list.get(i))
        name = image_names[i]
        
        typed_images = {"double": image.toDouble(), "float": image.toFloat(), "byte": image.toByte(), "int": image.toInt()}
        export_task = ee.batch.Export.image.toAsset(image = typed_images[data_type],
                                                      description = name,
                                                      assetId = asset_folder+"/"+name,
                                                      region = image.geometry(),
                                                      scale = scale,
                                                      maxPixels = max_pixels)
        export_task.start()
        task_list.append(export_task)
    
    return task_list

In [289]:
def export_collection_to_drive(collection, num_images: int = 0, image_names: list = [], gdrive_folder: str = "", scale: float = 20, max_pixels: int = 1e8, data_type: str = "float"):
    
    collection = ee.ImageCollection(collection)
    image_list = collection.toList(num_images)
    task_list = []

    for i in range(num_images):
        image = ee.Image(image_list.get(i))
        name = image_names[i]
        print(name)
        typed_images = {"double": image.toDouble(), "float": image.toFloat(), "byte": image.toByte(), "int": image.toInt()}
        export_task = ee.batch.Export.image.toDrive(image = typed_images[data_type],
                                                    description = name,
                                                    folder = gdrive_folder,
                                                    fileNamePrefix = name,
                                                    region = image.geometry(),
                                                    scale = scale,
                                                    maxPixels = max_pixels)
        export_task.start()
        task_list.append(export_task)
    
    return task_list

In [431]:
def s2_createFeatureCollection_estimates():
    return ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/s2_sl2p_weiss_or_prosail_NNT3_Single_0_1') \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/s2_sl2p_weiss_or_prosail_NNT3_Single_0_1'))   \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/s2_sl2p_ROF_sobol_prosail_dbf_big_clumped_NNT1_Single_0_1_v2'))  \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/s2_sl2p_ROF_sobol_prosail_enf_big_clumped_NNT1_Single_0_1_v2'))   \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/s2_sl2p_weiss_or_prosail_NNT3_Single_0_1'))  \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/s2_sl2p_weiss_or_prosail_NNT3_Single_0_1'))  \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/s2_sl2p_weiss_or_prosail_NNT3_Single_0_1'))  \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/s2_sl2p_ROF_sobol_prosail_dbf_big_clumped_NNT1_Single_0_1_v2'))   \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/s2_sl2p_weiss_or_prosail_NNT3_Single_0_1'))  \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/s2_sl2p_weiss_or_prosail_NNT3_Single_0_1')) \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/s2_sl2p_weiss_or_prosail_NNT3_Single_0_1')) \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/s2_sl2p_ROF_sobol_prosail_mix_big_clumped_NNT1_Single_0_1_v2'))   


In [473]:
def s2_createFeatureCollection_errors():
    return ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/s2_sl2p_weiss_or_prosail_NNT3_Single_0_1_error') \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/s2_sl2p_weiss_or_prosail_NNT3_Single_0_1_error')) \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/s2_sl2p_ROF_sobol_prosail_dbf_big_clumped_NNT1_Single_0_1_v2_errors')) \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/s2_sl2p_ROF_sobol_prosail_enf_big_clumped_NNT1_Single_0_1_v2_errors')) \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/s2_sl2p_weiss_or_prosail_NNT3_Single_0_1_error')) \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/s2_sl2p_weiss_or_prosail_NNT3_Single_0_1_error')) \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/s2_sl2p_weiss_or_prosail_NNT3_Single_0_1_error')) \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/s2_sl2p_ROF_sobol_prosail_dbf_big_clumped_NNT1_Single_0_1_v2_errors')) \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/s2_sl2p_weiss_or_prosail_NNT3_Single_0_1_error')) \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/s2_sl2p_weiss_or_prosail_NNT3_Single_0_1_error')) \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/s2_sl2p_weiss_or_prosail_NNT3_Single_0_1_error')) \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/s2_sl2p_ROF_sobol_prosail_mix_big_clumped_NNT1_Single_0_1_v2_errors'))


In [474]:
def s2_createFeatureCollection_domains():
    return ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/S2_SL2P_WEISS_ORIGINAL_DOMAIN') \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/S2_SL2P_WEISS_ORIGINAL_DOMAIN')) \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/S2_SL2P_WEISS_ORIGINAL_DOMAIN')) \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/S2_SL2P_WEISS_ORIGINAL_DOMAIN')) \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/S2_SL2P_WEISS_ORIGINAL_DOMAIN')) \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/S2_SL2P_WEISS_ORIGINAL_DOMAIN')) \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/S2_SL2P_WEISS_ORIGINAL_DOMAIN')) \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/S2_SL2P_WEISS_ORIGINAL_DOMAIN')) \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/S2_SL2P_WEISS_ORIGINAL_DOMAIN')) \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/S2_SL2P_WEISS_ORIGINAL_DOMAIN')) \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/S2_SL2P_WEISS_ORIGINAL_DOMAIN')) \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/S2_SL2P_WEISS_ORIGINAL_DOMAIN'))

In [475]:
def s2_createFeatureCollection_range():
    return ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/weiss_or_prosail3_NNT3_Single_0_1_RANGE') \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/weiss_or_prosail3_NNT3_Single_0_1_RANGE')) \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/weiss_or_prosail3_NNT3_Single_0_1_RANGE')) \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/weiss_or_prosail3_NNT3_Single_0_1_RANGE')) \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/weiss_or_prosail3_NNT3_Single_0_1_RANGE')) \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/weiss_or_prosail3_NNT3_Single_0_1_RANGE')) \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/weiss_or_prosail3_NNT3_Single_0_1_RANGE')) \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/weiss_or_prosail3_NNT3_Single_0_1_RANGE')) \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/weiss_or_prosail3_NNT3_Single_0_1_RANGE')) \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/weiss_or_prosail3_NNT3_Single_0_1_RANGE')) \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/weiss_or_prosail3_NNT3_Single_0_1_RANGE')) \
                    .merge(ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/weiss_or_prosail3_NNT3_Single_0_1_RANGE')) 

In [476]:
def s2_createFeatureCollection_Network_Ind():
    return ee.FeatureCollection('users/rfernand387/Parameter_file_prosail_ccrs_big_clumped2')

In [477]:
# land cover partition, we merge nalcms last so it takes precedence in a mosaic
def s2_createImageCollection_partition():
    return ee.ImageCollection("COPERNICUS/Landcover/100m/Proba-V/Global") \
             .map( lambda image: image.select("discrete_classification") \
             .remap([0,20,30,40,50,60,70,80,90,100,111,112,113,114,115,116,121,122,123,124,125,126,200],[0,8,10,15,17,16,19,18,14,13,1,3,1,5,6,6,2,4,2,5,6,6,18],0).toUint8().rename("partition")) \
             .merge(ee.ImageCollection('users/rfernand387/NA_NALCMS_2015_tiles') \
             .map(lambda image: image.select("b1").rename("partition")))

In [478]:
def s2_createFeatureCollection_legend():
    return ee.FeatureCollection('users/rfernand387/Legend_prosail_ccrs_big_clumped')

In [479]:
def l8_createFeatureCollection_estimates():
    return ee.FeatureCollection('users/rfernand387/LANDSAT_LC08_C01_T1_SR_SL2P_OUTPUT')

In [480]:
def l8_createFeatureCollection_errors():
    return ee.FeatureCollection('users/rfernand387/LANDSAT_LC08_C01_T1_SR_SL2P_ERRORS')

In [481]:
def l8_createFeatureCollection_domains():
    return ee.FeatureCollection('users/rfernand387/LANDSAT_LC08_C01_T1_SR/LANDSAT_LC08_C01_T1_SR_DOMAIN')

In [482]:
def l8_createFeatureCollection_range():
    return ee.FeatureCollection('users/rfernand387/LANDSAT_LC08_C01_T1_SR/LANDSAT_LC08_C01_T1_SR_RANGE')

In [483]:
def l8_createFeatureCollection_Network_Ind():
    return ee.FeatureCollection('users/rfernand387/LANDSAT_LC08_C01_T1_SR/Parameter_file_sl2p')

In [484]:
def l8_createImageCollection_partition():
    return ee.ImageCollection('users/rfernand387/NA_NALCMS_2015_tiles').map(lambda image: image.select("b1").rename("partition")).merge(ee.ImageCollection("COPERNICUS/Landcover/100m/Proba-V/Global").map( lambda image: image.select("discrete_classification").remap([0,20,30,40,50,60,70,80,90,100,111,112,113,114,115,116,121,122,123,124,125,126,200],[0,8,10,15,17,16,19,18,14,13,1,3,1,5,6,6,2,4,2,5,6,6,18],0).toUint8().rename("partition")))

In [485]:
def l8_createFeatureCollection_legend():
    return ee.FeatureCollection('users/rfernand387/LANDSAT_LC08_C01_T1_SR/Legend_sl2p')

In [486]:
# add a 'date' band: number of days since epoch
def addDate(image):
    return image.addBands(ee.Image.constant(ee.Date(image.date()).millis().divide(86400000)).rename('date').toUint16())

In [487]:
#computes a delta time property for an image
def deltaTime(midDate,image) :
    return ee.Image(image.set("deltaTime",ee.Number(image.date().millis()).subtract(ee.Number(midDate)).abs()))

In [488]:
# mask pixels that are not clear sky in a S2 MSI image
def s2MaskClear(image) :
    qa = image.select('QA60');
    mask = qa.bitwiseAnd(1<<10).eq(0).And(qa.bitwiseAnd(1<<11).eq(0));
    return image.updateMask(mask)

In [489]:
# add s2 geometry bands scaled by 10000
def addS2Geometry(colOptions,image) :
    return (image.addBands(image.metadata(colOptions["vza"]).multiply(3.1415).divide(180).cos().multiply(10000).toUint16().rename(['cosVZA']))
              .addBands(image.metadata(colOptions["sza"]).multiply(3.1415).divide(180).cos().multiply(10000).toUint16().rename(['cosSZA']))
              .addBands(image.metadata(colOptions["saa"]).subtract(image.metadata(colOptions["saa"])).multiply(3.1415).divide(180).cos().multiply(10000).toInt16().rename(['cosRAA'])));

In [490]:
# sentinel 2 land mask
def s2MaskLand(image) :
    #return image.updateMask(image.select('SCL').gt(0))
    return image.updateMask((image.select('SCL').eq(4)).Or(image.select('SCL').eq(5)))

In [491]:
# returns image with selected bands scaled
def scaleBands(bandList,scaleList,image) :
    bandList = ee.List(bandList)
    scaleList = ee.List(scaleList)
    return image.addBands(srcImg = image.select(bandList).multiply(ee.Image.constant(scaleList)).rename(bandList),overwrite = True)

In [492]:
# Determine if inputs fall in domain of algorithm
# Need to be updated to allow for the domain to vary with partition
def invalidInput(sl2pDomain,bandList,image) :
    
    sl2pDomain =  ee.FeatureCollection(sl2pDomain).aggregate_array("DomainCode").sort()
    bandList = ee.List(bandList).slice(3)
    image = ee.Image(image)

    # code image bands into a single band and compare to valid codes to make QC band
    image = image.addBands(image.select(bandList)
                              .multiply(ee.Image.constant(ee.Number(10)))
                              .ceil()
                              .mod(ee.Number(10))
                              .uint8()
                              .multiply(ee.Image.constant(ee.List.sequence(0,bandList.length().subtract(1)).map(lambda value: ee.Number(10).pow(ee.Number(value)))))
                              .reduce("sum")  
                              .remap(sl2pDomain, ee.List.repeat(0, sl2pDomain.length()),1)
                              .rename("QC"))
    return image




In [493]:
# returns image with single band named network id corresponding given 
def makeIndexLayer(image,legend,Network_Ind) :
    
    image = ee.Image(image)                        # partition image
    legend = ee.FeatureCollection(legend)          # legend to convert partition numbers to networks
    Network_Ind = ee.FeatureCollection(Network_Ind) # legend to convert networks to networkIDs
    
    #get lists of valid partitions
    legend_list = legend.toList(legend.size())
    landcover= legend_list.map(lambda feature: ee.Feature(feature).getNumber('Value'))

    # get corresponding networkIDs
    networkIDs = legend_list.map(lambda feature: ee.Feature(feature).get('SL2P Network')) \
                              .map(lambda propertyValue:  ee.Feature(ee.FeatureCollection(Network_Ind).first()).toDictionary().getNumber(propertyValue))
    
    return image.remap(landcover, networkIDs, 0).rename('networkID')

In [494]:
# Read coefficients of a network from csv EE asset
def getCoefs(netData,ind) :
    return((ee.Feature(netData)).getNumber(ee.String('tabledata').cat(ee.Number(ind).int().format())))

In [495]:
# Parse one row of CSV file for a network into a global variable
#
# We assume a two hidden layer network with tansig functions but
# allow for variable nodes per layer
def makeNets(feature, M) :
    
    feature = ee.List(feature);
    M = ee.Number(M);
    
    # get the requested network and initialize the created network
    netData = ee.Feature(feature.get(M.subtract(1)));
    net = {};
    
    # input slope
    num = ee.Number(6);
    start = num.add(1);
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())));
    net["inpSlope"] = ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind))
    
    # input offset
    num = end.add(1)
    start = num.add(1)
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())))
    net["inpOffset"] = ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind))

    # hidden layer 1 weight
    num = end.add(1)
    start = num.add(1)
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())))
    net["h1wt"] = ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind))

    # hidden layer 1 bias
    num = end.add(1)
    start = num.add(1)
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())))
    net["h1bi"] = ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind))

    # hidden layer 2 weight
    num = end.add(1)
    start = num.add(1)
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())))
    net["h2wt"] = ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind))
  
    # hidden layer 2 bias
    num = end.add(1)
    start = num.add(1)
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())))
    net["h2bi"] = ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind))

    # output slope
    num = end.add(1)
    start = num.add(1)
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())))
    net["outSlope"] = ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind))
  
    # output offset
    num = end.add(1)
    start = num.add(1)
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())))
    net["outBias"] = ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind))
    
    return(ee.Dictionary(net))


In [496]:
# Parse CSV file with list of networks for a selected variable  
#This will parse one network for each landclass partition
def makeNetVars(asset, numNets, variableNum) :
    
    asset= ee.FeatureCollection(asset)
    numNets = ee.Number(numNets)
    variableNum= ee.Number(variableNum)  

    #get selected network 
    list_features = asset.flatten()
    filtered_features = ee.FeatureCollection(asset.filter(ee.Filter.eq('tabledata3', variableNum))).toList(numNets)
    
    return ee.List.sequence(1,numNets).map(lambda netNum: makeNets(filtered_features,netNum))


In [497]:
# applies two layer neural network within input and output scaling
def applyNet(outputName,netDict) :
    
    outputName = ee.String(outputName)
    netDict = ee.Dictionary(netDict)
    
    inp = ee.Image(netDict.get('Image'))
    net = ee.Dictionary(netDict.get('Network'))
    
    # Input scaling
    l1inp2D = inp.multiply(ee.Image(net.toArray(ee.List(['inpSlope']),0).transpose()) \
                                          .arrayProject([0]) \
                                          .arrayFlatten([inp.bandNames()])) \
                   .add(ee.Image(net.toArray(ee.List(['inpOffset']),0).transpose()) \
                                          .arrayProject([0]) \
                                          .arrayFlatten([inp.bandNames()]))
    
    # Hidden layers
    l12D = ee.Image(net.toArray(ee.List(['h1wt']),0).reshape([ee.List(net.get('h1bi')).length(),ee.List(net.get('inpOffset')).length()])) \
              .matrixMultiply(l1inp2D.toArray().toArray(1)) \
              .add(ee.Image(net.toArray(ee.List(['h1bi']),0).transpose())) \
              .arrayProject([0]).arrayFlatten([['h1w1','h1w2','h1w3','h1w4','h1w5']])
    
    # apply tansig 2/(1+exp(-2*n))-1
    l2inp2D = ee.Image(2).divide(ee.Image(1).add((ee.Image(-2).multiply(l12D)).exp())).subtract(ee.Image(1))
    
    # purlin hidden layers
    l22D = l2inp2D.multiply(ee.Image(net.toArray(ee.List(['h2wt']),0).transpose()) \
                                          .arrayProject([0]) \
                                          .arrayFlatten([['h2w1','h2w2','h2w3','h2w4','h2w5']])) \
                    .reduce('sum') \
                    .add(ee.Image(net.toArray(ee.List(['h2bi']),0))) \
                                          .arrayProject([0]) \
                                          .arrayFlatten([['h2bi']])
    
    # Output scaling 
    outputBand = l22D.subtract(ee.Image(ee.Number(net.get('outBias')))) \
                    .divide(ee.Image(ee.Number(net.get('outSlope')))) 
    
    # Return network output
    return (outputBand.rename(outputName))

In [498]:
# returns image with single band named networkid corresponding given 
# input partition image remapped to networkIDs
# applies a set of shallow networks to an image based on a provided partition image band
def wrapperNNets(network, partition, netOptions, colOptions, layerName, imageInput) :

    # typecast function parameters
    network = ee.List(network)
    partition = ee.Image(partition)
    netOptions = netOptions
    colOptions = colOptions
    layerName = layerName
    imageInput = ee.Image(imageInput)

    # parse partition  used to identify network to use
    partition = partition.clip(imageInput.geometry()).select(['partition'])
    #.remap([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19],[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,15,15,15,15],0).rename('partition')

    # determine networks based on collection
    netList = ee.List(network.get(ee.Number(netOptions.get("variable")).subtract(1))); 
    
    # parse land cover into network index and add to input image
    imageInput = imageInput.addBands(makeIndexLayer(partition,colOptions["legend"],colOptions["Network_Ind"]))

    # define list of input names
    return ee.ImageCollection(ee.List.sequence(0, netList.size().subtract(1)) \
                                                    .map(lambda netIndex: selectNet(imageInput,netList,netOptions.get("inputBands"),netIndex)) \
                                                    .map(lambda netDict: applyNet(layerName,netDict))) \
                                .max() \
                  .addBands(partition) \
                  .addBands(imageInput.select('networkID'))

In [499]:
# applies two layer neural network within input and output scaling
def applyNet2(outputName,netDict) :
    
    print("applyNet2")
    print(netDict.getInfo())
    outputName = ee.String(outputName)
    netDict = ee.Dictionary(netDict)
    inp = ee.Image(netDict.get('Image'))
    net = ee.Dictionary(netDict.get('Network'))
    print(net.getInfo())
    # Input scaling
    l1inp2D = inp.multiply(ee.Image(net.toArray(ee.List(['inpSlope']),0).transpose()) \
                                          .arrayProject([0]) \
                                          .arrayFlatten([inp.bandNames()])) \
                   .add(ee.Image(net.toArray(ee.List(['inpOffset']),0).transpose()) \
                                          .arrayProject([0]) \
                                          .arrayFlatten([inp.bandNames()]))

    # Hidden layers
    l12D = ee.Image(net.toArray(ee.List(['h1wt']),0).reshape([ee.List(net.get('h1bi')).length(),ee.List(net.get('inpOffset')).length()])) \
              .matrixMultiply(l1inp2D.toArray().toArray(1)) \
              .add(ee.Image(net.toArray(ee.List(['h1bi']),0).transpose())) \
              .arrayProject([0]).arrayFlatten([['h1w1','h1w2','h1w3','h1w4','h1w5']])
    
    # apply tansig 2/(1+exp(-2*n))-1
    l2inp2D = ee.Image(2).divide(ee.Image(1).add((ee.Image(-2).multiply(l12D)).exp())).subtract(ee.Image(1))
    
    # purlin hidden layers
    l22D = l2inp2D.multiply(ee.Image(net.toArray(ee.List(['h2wt']),0).transpose()) \
                                          .arrayProject([0]) \
                                          .arrayFlatten([['h2w1','h2w2','h2w3','h2w4','h2w5']])) \
                    .reduce('sum') \
                    .add(ee.Image(net.toArray(ee.List(['h2bi']),0))) \
                                          .arrayProject([0]) \
                                          .arrayFlatten([['h2bi']])
    
    # Output scaling 
    outputBand = l22D.subtract(ee.Image(ee.Number(net.get('outBias')))) \
                    .divide(ee.Image(ee.Number(net.get('outSlope')))) 

    # Return network output
    return (outputBand.rename(outputName))

In [500]:
# Read coefficients of a network from csv EE asset
def getCoefs2(netData,ind) :
    return((ee.Feature(netData)).getNumber(ee.String('tabledata').cat(ee.Number(ind).int().format())))

In [501]:
# Parse one row of CSV file for a network into a global variable
#
# We assume a two hidden layer network with tansig functions but
# allow for variable nodes per layer
def makeNets2(feature, M) :
    
    feature = ee.List(feature);
    M = ee.Number(M);
    
    # get the requested network and initialize the created network
    netData = ee.Feature(feature.get(M.subtract(1)));
    net = ee.Dictionary();
    
    # input slope
    num = ee.Number(6);
    start = num.add(1);
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())));
    net = net.set("inpSlope", ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind)))
    
    # input offset
    num = end.add(1)
    start = num.add(1)
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())));
    net = net.set("inpOffset", ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind)))

    # hidden layer 1 weight
    num = end.add(1)
    start = num.add(1)
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())));
    net = net.set("h1wt", ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind)))

    # hidden layer 1 bias
    num = end.add(1)
    start = num.add(1)
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())));
    net = net.set("h1bi", ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind)))

    # hidden layer 2 weight
    num = end.add(1)
    start = num.add(1)
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())));
    net = net.set("h2wt", ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind)))
  
    # hidden layer 2 bias
    num = end.add(1)
    start = num.add(1)
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())));
    net = net.set("h2bi", ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind)))

    # output slope
    num = end.add(1)
    start = num.add(1)
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())));
    net = net.set("outSlope", ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind)))
  
    # output offset
    num = end.add(1)
    start = num.add(1)
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())));
    net = net.set("outBias", ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind)))

    return(net)


In [502]:
# Parse CSV file with list of networks for a selected variable  
#This will parse one network for each landclass partition
def makeNetVars2(asset, numNets, variableNum) :
    
    print("makenetVars2")
    asset= ee.FeatureCollection(asset)
    numNets = ee.Number(numNets)
    variableNum= ee.Number(variableNum)
    

    #get selected network 
    filtered_features = ee.FeatureCollection(asset.filter(ee.Filter.eq('tabledata3', variableNum))).toList(numNets)
    
    # make only first net
    netList = makeNets2(filtered_features,1)
    return netList
    #return ee.List.sequence(1,numNets).map(lambda netNum: makeNets(filtered_features,netNum))


In [503]:
COLLECTION_OPTIONS = {
    'COPERNICUS/S2_SR': {
      "name": 'S2',
      "description": 'Sentinel 2A',
      "Cloudcover": 'CLOUDY_PIXEL_PERCENTAGE',
      "Watercover": 'WATER_PERCENTAGE',
      "sza": 'MEAN_SOLAR_ZENITH_ANGLE',
      "vza": 'MEAN_INCIDENCE_ZENITH_ANGLE_B8A',
      "saa": 'MEAN_SOLAR_AZIMUTH_ANGLE', 
      "vaa": 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A',
      "VIS_OPTIONS": 'VIS_OPTIONS',
      "Collection_SL2P": ee.FeatureCollection(s2_createFeatureCollection_estimates()),
      "Collection_SL2Perrors": ee.FeatureCollection(s2_createFeatureCollection_errors()),  
      "sl2pDomain": ee.FeatureCollection(s2_createFeatureCollection_domains()),
      "Network_Ind": ee.FeatureCollection(s2_createFeatureCollection_Network_Ind()),
      "partition": ee.ImageCollection(s2_createImageCollection_partition()),
      "legend":  ee.FeatureCollection(s2_createFeatureCollection_legend()),
      "numVariables": 7
    },
    'LANDSAT/LC08/C01/T1_SR': {
      "name": 'L8',
      "description": 'LANDSAT 8',
      "Cloudcover": 'CLOUD_COVER_LAND',
      "Watercover": 'CLOUD_COVER',
      "sza": 'SOLAR_ZENITH_ANGLE',
      "vza": 'SOLAR_ZENITH_ANGLE',
      "saa": 'SOLAR_AZIMUTH_ANGLE', 
      "vaa": 'SOLAR_AZIMUTH_ANGLE',
      "VIS_OPTIONS": 'VIS_OPTIONS',
      "Collection_SL2P": ee.FeatureCollection(l8_createFeatureCollection_estimates()),
      "Collection_SL2Perrors": ee.FeatureCollection(l8_createFeatureCollection_errors()),
      "sl2pDomain": ee.FeatureCollection(l8_createFeatureCollection_domains()),
      "Network_Ind": ee.FeatureCollection(l8_createFeatureCollection_Network_Ind()),
      "partition": ee.ImageCollection(l8_createImageCollection_partition()),
      "legend":  ee.FeatureCollection(l8_createFeatureCollection_legend()),
      "numVariables": 7
    }
}
VIS_OPTIONS = {
    "Surface_Reflectance": {
        "COPERNICUS/S2_SR": {
            "Name": 'Surface_Reflectance',
            "description": 'Surface_Reflectance',
            "inp":      [ 'B4', 'B5', 'B6', 'B7', 'B8A','B9','B11','B12']
        }
      },
    "Albedo": {
        "COPERNICUS/S2_SR": {
            "Name": 'Albedo',
            "errorName": 'errorAlbedo',
            "maskName": 'maskAlbedo',
            "description": 'Black sky albedo',
            "variable": 6,
            "inputBands":      [ 'cosVZA','cosSZA','cosRAA','B3','B4', 'B5', 'B6', 'B7', 'B8A','B11','B12'],
            "inputScaling":      [0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
        }
    },
    'fAPAR': {
        "COPERNICUS/S2_SR": {
            "Name": 'fAPAR',
            "errorName": 'errorfAPAR',
            "maskName": 'maskfAPAR',
            "description": 'Fraction of absorbed photosynthetically active radiation',
            "variable": 2,
            "inputBands":      [ 'cosVZA','cosSZA','cosRAA','B3','B4', 'B5', 'B6', 'B7', 'B8A','B11','B12'],
            "inputScaling":      [0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
      }
    },
    'fCOVER': {
        "COPERNICUS/S2_SR": {
            "Name": 'fCOVER',
            "errorName": 'errorfCOVER',
            "maskName": 'maskfCOVER',
            "description": 'Fraction of canopy cover',
            "variable": 3,
            "inputBands":      [ 'cosVZA','cosSZA','cosRAA','B3','B4', 'B5', 'B6', 'B7', 'B8A','B11','B12'],
            "inputScaling":      [0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]]))) 
      }
    },
    'LAI': {
        "COPERNICUS/S2_SR": {
            "Name": 'LAI',
            "errorName": 'errorLAI',
            "maskName": 'maskLAI',
            "description": 'Leaf area index',
            "variable": 1,
            "inputBands":      [ 'cosVZA','cosSZA','cosRAA','B3','B4', 'B5', 'B6', 'B7', 'B8A','B11','B12'],
            "inputScaling":      [0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
      }  
    },
    'CCC': {
        "COPERNICUS/S2_SR": {
            "Name": 'CCC',
            "errorName": 'errorCCC',
            "maskName": 'maskCCC',
            "description": 'Canopy chlorophyll content',
            "variable": 4,
            "inputBands":      [ 'cosVZA','cosSZA','cosRAA','B3','B4', 'B5', 'B6', 'B7', 'B8A','B11','B12'],
            "inputScaling":      [0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1000]])))
      } 
    },
    'CWC': {
        "COPERNICUS/S2_SR": {
            "Name": 'CWC',
            "errorName": 'errorCWC',
            "maskName": 'maskCWC',
            "description": 'Canopy water content',
            "variable": 5,
            "inputBands":      [ 'cosVZA','cosSZA','cosRAA','B3','B4', 'B5', 'B6', 'B7', 'B8A','B11','B12'],
            "inputScaling":      [0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[100]])))
      } 
    },
    'DASF': {
        "COPERNICUS/S2_SR": {
            "Name": 'DASF',
            "errorName": 'errorDASF',
            "maskName": 'maskDASF',
            "description": 'Directional area scattering factor',
            "variable": 7,
            "inputBands":      [ 'cosVZA','cosSZA','cosRAA','B3','B4', 'B5', 'B6', 'B7', 'B8A','B11','B12'],
            "inputScaling":      [0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
      } 
    }
}

In [504]:
# returns lists of sampled values for each band in an image as a new feature property
def sampleProduct(image, scaleSample, filterSize,sampleRegion, sampleName) :
    
    # cast vars
    image = ee.Image(image)
    scaleSample= ee.Number(scaleSample)
    sampleRegion = ee.Feature(sampleRegion)
    filterSize = ee.Number(filterSize)
    
    # Define a boxcar or low-pass kernel.
    boxcar = ee.Kernel.square(radius= filterSize, units= 'meters', normalize= True);

    # produce feature collection where each feature is a list of samples from a given band
    sampleData = image.convolve(boxcar).sample(region=sampleRegion.geometry(), projection=image.projection(), scale=scaleSample ,geometries=True, dropNulls = False)
    #print(sampleData)
    # for each band get a dictionary of sampled values as a property of the sampleRegion feature
    
    sampleList= ee.List(image.bandNames().map(lambda bandName: ee.Dictionary({ 'bandName': bandName, 'data': sampleData.aggregate_array(bandName)})))
    
    return sampleRegion.set(sampleName,sampleList)

In [505]:
# returns lists of sampled values for each band in an image as a new feature property
def sampleProductCollection2(productCollection, outputScale, filterSize,sampleRegion, sampleName) :

    print('sampleProductCollection2')
    productCollection = ee.ImageCollection(productCollection)
    outputScale= ee.Number(outputScale)
    sampleRegion = ee.Feature(sampleRegion)
    filterSize = ee.Number(filterSize)
    
    # Define a boxcar or low-pass kernel.
    boxcar = ee.Kernel.square(radius= filterSize, units= 'meters', normalize= True);

    # produce feature collection where each feature a feature collectiion corresponding to a list of samples from a given band from one product image
    sampleData = productCollection.map(lambda image: image.convolve(boxcar).sample(region=sampleRegion.geometry(), projection=image.select('date').projection(), scale=outputScale,geometries=True, dropNulls = False) ).flatten()
    #print('sampleData',sampleData.first().getInfo())
    
    # for each band get a dictionary of sampled values as a property of the sampleRegion feature
    sampleList= ee.List(productCollection.first().bandNames().map(lambda bandName: ee.Dictionary({ 'bandName': bandName, 'data': sampleData.aggregate_array(bandName)})))
    
    return sampleRegion.set(sampleName,sampleList)

In [506]:
def makeProduct(mapBounds ,colName,outputName,startDate,endDate,maxCloudcover,outputScale,outputOffset) :

    # parse the networks
    colOptions = COLLECTION_OPTIONS[colName]
    netOptions = VIS_OPTIONS[outputName][colName]
    numNets = ee.Number(ee.Feature((COLLECTION_OPTIONS[colName]["Network_Ind"]).first()).propertyNames().remove('Feature Index').remove('system:index').remove('lon').size())
    #SL2P1 = makeNetVars2(COLLECTION_OPTIONS[colName]["Collection_SL2P"],numNets,1)
    SL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2P"],numNets,netNum));
    errorsSL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2Perrors"],numNets,netNum));
    
    # make products 
    input_collection =  ee.ImageCollection(colName) \
                          .filterBounds(mapBounds) \
                          .filterDate(startDate, endDate) \
                          .filterMetadata(colOptions["Cloudcover"],'less_than',maxCloudcover) \
                          .limit(5000) \
                          .map(lambda image: addDate(image)) \
                          .map(lambda image: image.clip(mapBounds)) \
                          .map(lambda image: s2MaskClear(image)) \
                          .map(lambda image: addS2Geometry(colOptions,image))
    print(input_collection.size().getInfo())

    if outputName == "Surface_Reflectance":
        products = input_collection;
    else:
        # get partition used to select network
        partition = (COLLECTION_OPTIONS[colName]["partition"]).filterBounds(mapBounds).mosaic().clip(mapBounds).rename('partition');
        # pre process input imagery and flag invalid inputs
        input_collection  =  input_collection.map(lambda image: s2MaskLand(image)) \
                                             .map(lambda image: scaleBands(netOptions["inputBands"],netOptions["inputScaling"],image)) \
                                             .map(lambda image: invalidInput(COLLECTION_OPTIONS[colName]["sl2pDomain"],netOptions["inputBands"],image))

        ## apply networks to produce mapped parameters
        estimateSL2P = input_collection.map(lambda image: wrapperNNets(SL2P,partition, netOptions, COLLECTION_OPTIONS[colName],"estimate"+outputName,image))
        uncertaintySL2P = input_collection.map(lambda image: wrapperNNets(errorsSL2P,partition, netOptions, COLLECTION_OPTIONS[colName],"error"+outputName,image))
        # scale and offset mapped parameter bands
        estimateSL2P = estimateSL2P.map(lambda image: image.addBands(image.select("estimate"+outputName).multiply(ee.Image.constant(outputScale)).add(ee.Image.constant(outputOffset)),overwrite=True));
        uncertaintySL2P = uncertaintySL2P.map(lambda image: image.addBands(image.select("error"+outputName).multiply(ee.Image.constant(outputScale)).add(ee.Image.constant(outputOffset)),overwrite=True));
        # produce final collection
        products =  input_collection.select(['date','QC']).combine(estimateSL2P).combine(uncertaintySL2P)
        
    return products

In [507]:
# returns dictionary with image masked so the networkID band equals the netIndex and the corresponding network
def selectNet2(image,netList,inputNames,netIndex) :
    
    image = ee.Image(image)
    netList = ee.List(netList)
    inputNames = ee.List(inputNames)
    netIndex = ee.Number(netIndex).int()
    print('selectedNet2',netList.size().getInfo(),netIndex.getInfo())
    test = ee.Dictionary()  \
            .set("Image", ee.Image(image.updateMask(image.select('networkID').eq(netIndex)).select(inputNames))) \
            .set("Network", ee.List(netList.get(netIndex)))
    
    result = ee.Dictionary()  \
            .set("Image", ee.Image(image.updateMask(image.select('networkID').eq(netIndex)).select(inputNames))) \
            .set("Network", ee.List(netList.get(netIndex)))
    #print('****',result.getInfo())

    return result

In [508]:
# returns dictionary with image masked so the networkID band equals the netIndex and the corresponding network
def selectNet(image,netList,inputNames,netIndex) :
    
    image = ee.Image(image)
    netList = ee.List(netList)
    inputNames = ee.List(inputNames)
    netIndex = ee.Number(netIndex).int()

    return ee.Dictionary()  \
            .set("Image", ee.Image(image.updateMask(image.select('networkID').eq(netIndex)).select(inputNames))) \
            .set("Network", ee.List(netList.get(netIndex))) 

In [509]:
# returns image with single band named network id corresponding given 
def makeIndexLayer2(image,legend,Network_Ind) :
    
    print("makeIndexLayer2")
    image = ee.Image(image)                        # partition image
    legend = ee.FeatureCollection(legend)          # legend to convert partition numbers to networks
    Network_Ind = ee.FeatureCollection(Network_Ind) # legend to convert networks to networkIDs
    
    #get lists of valid partitions
    legend_list = legend.toList(legend.size())
    landcover= legend_list.map(lambda feature: ee.Feature(feature).getNumber('Value'))

    # get corresponding networkIDs
    networkIDs = legend_list.map(lambda feature: ee.Feature(feature).get('SL2P Network')) \
                              .map(lambda propertyValue:  ee.Feature(ee.FeatureCollection(Network_Ind).first()).toDictionary().getNumber(propertyValue))
    #print(networkIDs.getInfo())
    return image.remap(landcover, networkIDs, 0).rename('networkID')

In [510]:
# returns image with single band named networkid corresponding given 
# input partition image remapped to networkIDs
# applies a set of shallow networks to an image based on a provided partition image band
def wrapperNNets2(network, partition, netOptions, colOptions, layerName, imageInput) :

    # typecast function parameters
    network = ee.List(network)
    partition = ee.Image(partition)
    netOptions = ee.Dictionary(netOptions)
    colOptions = ee.Dictionary(colOptions)
    layerName = ee.String(layerName)
    imageInput = ee.Image(imageInput)

    # parse partition  used to identify network to use
    partition = partition.clip(imageInput.geometry()).select(['partition'])

    # determine networks based on collection
    netList = ee.List(network.get(ee.Number(netOptions.get("variable")).subtract(1))) 
    #print(netList.getInfo())
    
    # parse land cover into network index and add to input image
    imageInput = imageInput.addBands(makeIndexLayer2(partition,colOptions.get("legend"),colOptions.get("Network_Ind")))
    #print(imageInput.getInfo())
    print('done selectNet2')
    
    #test = (ee.List.sequence(0, netList.size().subtract(1)) \
    #                                                .map(lambda netIndex: selectNet(imageInput,netList,netOptions.get("inputBands"),netIndex)))
    #print('test',test.getInfo())
    return ee.ImageCollection(ee.List.sequence(0, netList.size().subtract(1)) \
                                                    .map(lambda netIndex: selectNet(imageInput,netList,netOptions.get("inputBands"),netIndex)) \
                                                    .map(lambda netDict: applyNet(layerName,netDict))) \
                                .max() \
                  .addBands(partition) \
                  .addBands(imageInput.select('networkID'))
    # define list of input names
    #netDict = ee.Dictionary(selectNet2(imageInput,netList,netOptions.get("inputBands"),netIndex));
    #estimate = ee.Image(applyNet2(layerName,netDict))

    #return estimate.addBands(partition).addBands(imageInput.select('networkID'))

In [511]:
#makes products for specified region and time period 
def makeProductCollection(collectionName,outputName,mapBounds,startDate,endDate,maxCloudcover) :

    print('makeProductCollection')
    # parse the networks
    colOptions = COLLECTION_OPTIONS[collectionName]
    netOptions = VIS_OPTIONS[outputName][collectionName]
    numNets = ee.Number(ee.Feature((COLLECTION_OPTIONS[collectionName]["Network_Ind"]).first()).propertyNames().remove('Feature Index').remove('system:index').remove('lon').size())
    SL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[collectionName]["numVariables"]),1).map(lambda netNum: makeNetVars(COLLECTION_OPTIONS[collectionName]["Collection_SL2P"],numNets,netNum))
    errorsSL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[collectionName]["numVariables"]),1).map(lambda netNum: makeNetVars(COLLECTION_OPTIONS[collectionName]["Collection_SL2Perrors"],numNets,netNum))

    # make products 
    input_collection =  ee.ImageCollection(collectionName) \
                      .filterBounds(mapBounds) \
                      .filterDate(startDate, endDate) \
                      .filterMetadata(colOptions["Cloudcover"],'less_than',maxCloudcover) \
                      .limit(5000) 
    
    # check if there are products
    products = []
    if input_collection.size().getInfo() >  0 :


        input_collection = input_collection.map(lambda image: addDate(image)).map(lambda image: image.clip(mapBounds)) 

        if collectionName=='COPERNICUS/S2_SR':
            input_collection =  input_collection.map(lambda image: s2MaskClear(image)).map(lambda image: addS2Geometry(colOptions,image))
        elif collectionName=='LANDSAT/LC08/C01/T1_SR': 
            input_collection =  input_collection.map(lambda image: L8MaskClear(image)).map(lambda image: addL8Geometry(colOptions,image))
        elif  collectionName=='users/rfernand387/L2avalidation': 
            input_collection = input_collection.map(lambda image: ccrsSentinel2(image)).map(lambda image: s2MaskClear(image)).map(lambda image: addS2Geometry(colOptions,image))
        else:
            print('Invalid collection')

        if outputName == "Surface_Reflectance":
            products = input_collection;
        else:
            # get partition used to select network
            partition = (COLLECTION_OPTIONS[collectionName]["partition"]).filterBounds(mapBounds).mosaic().clip(mapBounds).rename('partition');
            # pre process input imagery and flag invalid inputs
            input_collection  =  input_collection.map(lambda image: s2MaskLand(image)) \
                                                 .map(lambda image: scaleBands(netOptions["inputBands"],netOptions["inputScaling"],image)) \
                                                 .map(lambda image: invalidInput(COLLECTION_OPTIONS[collectionName]["sl2pDomain"],netOptions["inputBands"],image))
            #print(input_collection.first().getInfo())
            test = wrapperNNets2(SL2P,partition, netOptions, COLLECTION_OPTIONS[collectionName],"estimate",input_collection.first())
            print(test.getInfo())
            print('******************************')
            ## apply networks to produce mapped parameters
            estimateSL2P = input_collection.map(lambda image: wrapperNNets(SL2P,partition, netOptions, COLLECTION_OPTIONS[collectionName],"estimate",image))
            uncertaintySL2P = input_collection.map(lambda image: wrapperNNets(errorsSL2P,partition, netOptions, COLLECTION_OPTIONS[collectionName],"error",image))

            # produce final collection
            products =  input_collection.select(['date','QC']).combine(estimateSL2P).combine(uncertaintySL2P)



        
    return products


In [512]:
# add dictionary of sampled values from product to a feature
def getSamples(site,outputName,collectionName,maxCloudcover,bufferSize,filterSize,outputScale,deltaTime):
    
    print('getSamples',outputName)
    # define vars
    site = ee.Feature(site)

    
    # get start and end date for this feature
    startDate = ee.Date(site.get('system:time_start')).advance(deltaTime[0],'day')
    endDate = ee.Date(site.get('system:time_start')).advance(deltaTime[1],'day')
    
    # make collection
    sampleDictionary = [];
    productCollection = []
    productCollection = makeProductCollection(collectionName,outputName,site.buffer(bufferSize+filterSize).geometry(),startDate,endDate,maxCloudcover)
    if productCollection :
        sampleDictionary = sampleProductCollection2(productCollection, outputScale, filterSize, site.buffer(bufferSize).geometry(),"sample" + outputName)
    return  sampleDictionary 

In [513]:
# make and sample products for CCRSInSituLAI2020   
# features to make product over product (should be larger than plotFeatures to allow for smoothing)
collectionName = "COPERNICUS/S2_SR"
maxCloudcover = 90
bufferSize = 60 # buffer around site geometry to sample
filterSize = 60 # length of square filter window applied to product
scaleSize = 20  # scale to produce and sample product
deltaTime = [-5,5] # number of days before and after sample

# define sample records
sampleRecords =  ee.FeatureCollection("users/hemitshah/GBOV/COPERNICUS_GBOV_RM7_20210313").filterDate('2019-01-01','2019-12-31').sort('system:time_start', False)
sampleRecords =  sampleRecords.toList(sampleRecords.size())
print(sampleRecords.size().getInfo())
# add match ups for requested products

# get GEE results in python dictionary
data_GBOV_RM6_20210313_LAI = []   
              


for n in range(0,sampleRecords.size().getInfo()) :  
    print(n)
    sampleDict = getSamples(sampleRecords.get(n),'LAI',collectionName,maxCloudcover,bufferSize,filterSize,scaleSize,deltaTime)
    if sampleDict :
        data_GBOV_RM6_20210313_LAI.append(ee.Dictionary(ee.Feature(sampleRecords.get(n)).toDictionary()).combine((ee.Dictionary(ee.Feature(sampleDict).toDictionary()))).getInfo())   
    #sampleDict = getSamples(sampleRecords.get(n),'fCOVER',collectionName,maxCloudcover,bufferSize,filterSize,scaleSize,deltaTime)
    #if sampleDict :
    #    data_GBOV_RM6_20210313_fCOVER.append((ee.Dictionary(ee.Feature(sampleDict).toDictionary())).getInfo())    
    #sampleDict = getSamples(sampleRecords.get(n),'fAPAR',collectionName,maxCloudcover,bufferSize,filterSize,scaleSize,deltaTime)
    #if sampleDict :
    #    data_GBOV_RM6_20210313_fAPAR.append((ee.Dictionary(ee.Feature(sampleDict).toDictionary())).getInfo()) 

1176
0
getSamples LAI
makeProductCollection
makeIndexLayer2
done selectNet2
{'type': 'Image', 'bands': [{'id': 'estimate', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'partition', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [1, 1], 'origin': [-82, 28], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'networkID', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 11}, 'dimensions': [1, 1], 'origin': [-82, 28], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
******************************
sampleProductCollection2
1
getSamples LAI
makeProductCollection
makeIndexLayer2
done selectNet2
{'type': 'Image', 'bands': [{'id': 'estimate', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'partition', 'data_type': {'type': 'PixelType', 'precision': 'in

In [418]:
# make and sample products for CCRSInSituLAI2020   
# features to make product over product (should be larger than plotFeatures to allow for smoothing)
collectionName = "COPERNICUS/S2_SR"
maxCloudcover = 90
bufferSize = 60 # buffer around site geometry to sample
filterSize = 60 # length of square filter window applied to product
scaleSize = 20  # scale to produce and sample product
deltaTime = [-5,5] # number of days before and after sample

# define sample records
sampleRecords =  ee.FeatureCollection("users/hemitshah/GBOV/COPERNICUS_GBOV_RM6_20210313").filterDate('2019-01-01','2019-12-31').sort('system:time_start', False)
sampleRecords =  sampleRecords.toList(sampleRecords.size())
print(sampleRecords.size().getInfo())
# add match ups for requested products

# get GEE results in python dictionary
data_GBOV_RM6_20210313_fAPAR = []   
              


for n in range(0,sampleRecords.size().getInfo()) :  
    print(n)
    sampleDict = getSamples(sampleRecords.get(n),'fAPAR',collectionName,maxCloudcover,bufferSize,filterSize,scaleSize,deltaTime)
    if sampleDict :
        data_GBOV_RM6_20210313_fAPAR.append(ee.Dictionary(ee.Feature(sampleRecords.get(n)).toDictionary()).combine((ee.Dictionary(ee.Feature(sampleDict).toDictionary()))).getInfo())   
    #sampleDict = getSamples(sampleRecords.get(n),'fCOVER',collectionName,maxCloudcover,bufferSize,filterSize,scaleSize,deltaTime)
    #if sampleDict :
    #    data_GBOV_RM6_20210313_fCOVER.append((ee.Dictionary(ee.Feature(sampleDict).toDictionary())).getInfo())    
    #sampleDict = getSamples(sampleRecords.get(n),'fAPAR',collectionName,maxCloudcover,bufferSize,filterSize,scaleSize,deltaTime)
    #if sampleDict :
    #    data_GBOV_RM6_20210313_fAPAR.append((ee.Dictionary(ee.Feature(sampleDict).toDictionary())).getInfo()) 

1176
0
getSamples fAPAR
makeProductCollection
makeIndexLayer2
done selectNet2
{'type': 'Image', 'bands': [{'id': 'estimate', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'partition', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [1, 1], 'origin': [-82, 28], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'networkID', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 11}, 'dimensions': [1, 1], 'origin': [-82, 28], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
******************************
sampleProductCollection2
1
getSamples fAPAR
makeProductCollection
makeIndexLayer2
done selectNet2
{'type': 'Image', 'bands': [{'id': 'estimate', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'partition', 'data_type': {'type': 'PixelType', 'precision':

In [419]:
# make and sample products for CCRSInSituLAI2020   
# features to make product over product (should be larger than plotFeatures to allow for smoothing)
collectionName = "COPERNICUS/S2_SR"
maxCloudcover = 90
bufferSize = 60 # buffer around site geometry to sample
filterSize = 60 # length of square filter window applied to product
scaleSize = 20  # scale to produce and sample product
deltaTime = [-5,5] # number of days before and after sample

# define sample records
sampleRecords =  ee.FeatureCollection("users/hemitshah/GBOV/COPERNICUS_GBOV_RM7_20210313").filterDate('2019-01-01','2019-12-31').sort('system:time_start', False)
sampleRecords =  sampleRecords.toList(sampleRecords.size())
print(sampleRecords.size().getInfo())
# add match ups for requested products

# get GEE results in python dictionary
data_GBOV_RM6_20210313_DASF = []   
              


for n in range(0,sampleRecords.size().getInfo()) :  
    print(n)
    sampleDict = getSamples(sampleRecords.get(n),'DASF',collectionName,maxCloudcover,bufferSize,filterSize,scaleSize,deltaTime)
    if sampleDict :
        data_GBOV_RM6_20210313_DASF.append(ee.Dictionary(ee.Feature(sampleRecords.get(n)).toDictionary()).combine((ee.Dictionary(ee.Feature(sampleDict).toDictionary()))).getInfo())   
    #sampleDict = getSamples(sampleRecords.get(n),'fCOVER',collectionName,maxCloudcover,bufferSize,filterSize,scaleSize,deltaTime)
    #if sampleDict :
    #    data_GBOV_RM6_20210313_fCOVER.append((ee.Dictionary(ee.Feature(sampleDict).toDictionary())).getInfo())    
    #sampleDict = getSamples(sampleRecords.get(n),'fAPAR',collectionName,maxCloudcover,bufferSize,filterSize,scaleSize,deltaTime)
    #if sampleDict :
    #    data_GBOV_RM6_20210313_fAPAR.append((ee.Dictionary(ee.Feature(sampleDict).toDictionary())).getInfo()) 

1176
0
getSamples DASF
makeProductCollection
makeIndexLayer2
done selectNet2
{'type': 'Image', 'bands': [{'id': 'estimate', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'partition', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [1, 1], 'origin': [-82, 28], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'networkID', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 11}, 'dimensions': [1, 1], 'origin': [-82, 28], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
******************************
sampleProductCollection2
1
getSamples DASF
makeProductCollection
makeIndexLayer2
done selectNet2
{'type': 'Image', 'bands': [{'id': 'estimate', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'partition', 'data_type': {'type': 'PixelType', 'precision': '

In [514]:
import pickle
with open("E:/wp3/Validation/data_GBOV_RM6_20210313_LAI_SL2PCCRS_v3.txt", "wb") as fp:   #Pickling
    pickle.dump(data_GBOV_RM6_20210313_LAI, fp)
with open("E:/wp3/Validation/data_GBOV_RM6_20210313_fAPAR_SL2PCCRS_v3.txt", "wb") as fp:   #Pickling
    pickle.dump(data_GBOV_RM6_20210313_fAPAR, fp)
with open("E:/wp3/Validation/data_GBOV_RM6_20210313_DASF_SL2PCCRS_v3.txt", "wb") as fp:   #Pickling
    pickle.dump(data_GBOV_RM6_20210313_DASF, fp)

In [429]:
data_GBOV_RM6_20210313_fAPAR[4]['samplefAPAR'][5]

{'bandName': 'error',
 'data': [0.614080613908144,
  0.6267090819477432,
  0.6186018794047918,
  0.6206409387219813,
  0.6308092965200539,
  0.6301154157471305,
  0.635062420600389,
  0.6396681729450078,
  0.6252503492040427,
  0.6359833605613996,
  0.6324562643706138,
  0.6315474282637663,
  0.6357405681624151,
  0.6400391825808539,
  0.6217654721327383,
  0.6263949674653593,
  0.6369658583299709,
  0.6310372895780835,
  0.6267172679714006,
  0.6282359704228102,
  0.6324630855124423,
  0.6142133048110088,
  0.6348230957360349,
  0.6252579009206493,
  0.6182757735310496,
  0.6169999155309578,
  0.6180030753113938,
  0.6065430995684774,
  0.6121002171631603,
  0.5964212568666177,
  0.5942980000862738,
  0.6060668438270542,
  0.5970804942033258,
  0.6043468377098917,
  0.611937037029277,
  0.6107530664981173,
  0.6139649479274923,
  0.6189604235773636,
  0.6109038097097474,
  0.6207191198978264,
  0.614878329208359,
  0.6130633307572626,
  0.6152130357889448,
  0.6193093806136434,
  0.60